In [1]:
!pip install rasa==1.10.5

In [2]:
!pip install -U tables==3.6.1

In [3]:
!pip install -U ipython

In [4]:
!python -m spacy download en

     |████████████████████████████████| 12.0 MB 22.7 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [5]:
!pip install nest_asyncio==1.3.3

In [6]:
import os
import rasa
import nest_asyncio

nest_asyncio.apply()
print("Event loop ready.")

Event loop ready.


In [7]:
from rasa.cli.scaffold import create_initial_project
project = "ChatBot-project"
create_initial_project(project)

In [8]:
# move into project directory and show files
os.chdir(project)
print(os.listdir("."))

['domain.yml', '__pycache__', 'credentials.yml', '__init__.py', 'tests', 'actions.py', 'endpoints.yml', 'config.yml', 'data']


In [9]:
config = "config.yml"
training_files = "data/"
domain = "domain.yml"
output = "models/"
print(config, training_files, domain, output)

config.yml data/ domain.yml models/


In [11]:
model_path = rasa.train(domain, config, [training_files], output)
print(model_path)

Training Core model...


Processed trackers: 100%|██████████| 5/5 [00:00<00:00, 83.44it/s, # actions=16]
Processed actions: 16it [00:00, 539.68it/s, # examples=16]
Epochs: 100%|██████████| 100/100 [00:16<00:00,  6.19it/s, t_loss=0.078, loss=0.006, acc=1.000]


Core model training completed.
Training NLU model...


Epochs: 100%|██████████| 100/100 [00:19<00:00,  5.10it/s, t_loss=1.673, i_loss=0.142, entity_loss=0.017, i_acc=1.000, entity_f1=0.844]


NLU model training completed.
Your Rasa model is trained and saved at '/content/ChatBot-project/models/20220427-131538.tar.gz'.
models/20220427-131538.tar.gz


In [12]:
from rasa.jupyter import chat

endpoints = 'endpoints.yml'

chat(model_path, endpoints)

Instructions for updating:
Use tf.identity instead.
Your bot is ready to talk! Type your messages here or send '/stop'.
Hey
Hey! How are you?
fine
Bye
i am sad
Here is something to cheer you up:


Did that help you?
yes
Great, carry on!
thank you
Bye
Bye
Bye
/stop


In [26]:
%%writefile data/nlu.md

## intent:report
- weather report of [London](area)
- report of [Delhi](area)
- [Mumbai](area) weather report
- [Dubai](area) weather
- [Paris](area)
- [California](area) weather
- [Bangkok](area) 
- [Rome](area) weather
- Wheather of [Cairo](area)?

## intent:greet
- hey
- hello
- hi
- good morning
- good evening
- hey there

## intent:goodbye
- bye
- goodbye
- see you around
- see you later

## intent:affirm
- yes
- indeed
- of course
- that sounds good
- correct

## intent:deny
- no
- never
- I don't think so
- don't like that
- no way
- not really

## intent:mood_great
- perfect
- very good
- great
- amazing
- wonderful
- I am feeling very good
- I am great
- I'm good

## intent:mood_unhappy
- sad
- very sad
- unhappy
- bad
- very bad
- awful
- terrible
- not very good
- extremely sad
- so sad

## intent:bot_challenge
- are you a bot?
- are you a human?
- am I talking to a bot?
- am I talking to a human?

Overwriting data/nlu.md


In [22]:
%%writefile domain.yml

intents:
  - greet
  - goodbye
  - affirm
  - deny
  - mood_great
  - mood_unhappy
  - bot_challenge
  - report

slots:
  area:
    type: unfeaturized

actions:
  - action_weather
    
responses:
  utter_greet:
  - text: "Hey! How may I help you?"

  utter_cheer_up:
  - text: "Here is something to cheer you up:"
    image: "https://i.imgur.com/nGF1K8f.jpg"

  utter_did_that_help:
  - text: "Did that help you?"

  utter_happy:
  - text: "Great, carry on!"

  utter_goodbye:
  - text: "Bye"

  utter_iamabot:
  - text: "I am a bot, powered by Rasa."

entities:
  - area

session_config:
  session_expiration_time: 60
  carry_over_slots_to_new_session: true

Overwriting domain.yml


In [23]:
%%writefile data/stories.md
  
## weather report
* greet
  - utter_greet
* report
  - action_weather
  
## happy path
* greet
  - utter_greet
* mood_great
  - utter_happy

## sad path 1
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* affirm
  - utter_happy

## sad path 2
* greet
  - utter_greet
* mood_unhappy
  - utter_cheer_up
  - utter_did_that_help
* deny
  - utter_goodbye

## say goodbye
* goodbye
  - utter_goodbye

## bot challenge
* bot_challenge
  - utter_iamabot

Overwriting data/stories.md


In [ ]:
!pip install pyowm

In [17]:
%%writefile actions.py
# This files contains your custom actions which can be used to run
# custom Python code.
#
# See this guide on how to implement these action:
# https://rasa.com/docs/rasa/core/actions/#custom-actions/


# This is a simple example for a custom action which utters "Hello World!"

from typing import Any, Text, Dict, List

from rasa_sdk import Action, Tracker
from rasa_sdk.executor import CollectingDispatcher
from pyowm import OWM
from rasa_sdk.events import SlotSet 


key = "c32fd41570da7f46432f516b654b03de"


# class ActionMap(Action):

#     def name(self) -> Text:
#         return "action_slot"

#     def run(self, dispatcher: CollectingDispatcher,
#             tracker: Tracker,
#             domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

#         if tracker.latest_message['entities'][0]['entity'] == 'area':
#             value = tracker.latest_message['entities'][0]['value']

#             return [SlotSet('new_area', value)]


class ActionReport(Action):

    def name(self) -> Text:
        return "action_weather"

    def run(self, dispatcher: CollectingDispatcher,
            tracker: Tracker,
            domain: Dict[Text, Any]) -> List[Dict[Text, Any]]:

        area = tracker.get_slot("area")
        owm = OWM(key)
        mgr = owm.weather_manager()

        observation = mgr.weather_at_place(area)
        w = observation.weather

        mess1 = f"weather of {area} is: "
        mess2 = f"Temperature is (in celsius): {w.temperature('celsius')['temp']}"
        mess3 = f"Wind speed is: {w.wind()['speed']}Km/h"
        mess4 = f"Humidity in {area} is: {w.humidity}%"
        mess5 = f"Overall weather status is: {w.detailed_status}."

        message = mess1 + '\n' + mess2 + '\n' + mess3 + '\n' + mess4 + '\n' + mess5
        dispatcher.utter_message(text=message)

        return []

Overwriting actions.py


In [27]:
model_path = rasa.train(domain, config, [training_files], output)
print(model_path)


Core stories/configuration did not change. No need to retrain Core model.
Training NLU model...


Epochs: 100%|██████████| 100/100 [00:20<00:00,  4.91it/s, t_loss=1.730, i_loss=0.201, entity_loss=0.012, i_acc=1.000, entity_f1=0.851]


NLU model training completed.
Your Rasa model is trained and saved at '/content/ChatBot-project/models/20220427-133726.tar.gz'.
models/20220427-133726.tar.gz


In [28]:
!rasa run actions & rasa shell

2022-04-27 13:37:36 INFO     rasa_sdk.endpoint  - Starting action endpoint server...
2022-04-27 13:37:36 INFO     rasa_sdk.executor  - Registered function for 'action_weather'.
2022-04-27 13:37:36 INFO     rasa_sdk.endpoint  - Action endpoint is up and running on http://localhost:5055
2022-04-27 13:37:37 INFO     root  - Generating grammar tables from /usr/lib/python3.7/lib2to3/Grammar.txt
2022-04-27 13:37:37 INFO     root  - Generating grammar tables from /usr/lib/python3.7/lib2to3/PatternGrammar.txt
2022-04-27 13:37:37 INFO     root  - Connecting to channel 'cmdline' which was specified by the '--connector' argument. Any other channels will be ignored. To connect to all given channels, omit the '--connector' argument.
2022-04-27 13:37:37 INFO     root  - Starting Rasa server on http://localhost:5005
2022-04-27 13:37:38.579481: E tensorflow/stream_executor/cuda/cuda_driver.cc:351] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-27 13:37:47 INFO 